In [1]:
import os
os.chdir('..')
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sklearn.utils import Bunch
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from aspectsent.metric import ClassificationReport

%load_ext autoreload 
%autoreload 2

# Linear SVM as baselines for Aspect detection and polarity detection 

## Read and explore

In [2]:
df_train = pd.read_csv('./Restaurants/sub_task_1/ABSA16_Restaurants_Train_SB1_v2.csv')
df_test = pd.read_csv('./Restaurants/sub_task_1/EN_REST_SB1_TEST_gold.csv')
df_train['category'] = df_train['category'].str.split('#').str[0]
df_test['category'] = df_test['category'].str.split('#').str[0]
df_train.head()

,rid,sid,sentence,category,target,from,to,polarity
0,1004293,1004293:0,Judging from previous posts this used to be a ...,RESTAURANT,place,51,56,negative
1,1004293,1004293:1,"We, there were four of us, arrived at noon - t...",SERVICE,staff,75,80,negative
2,1004293,1004293:2,"They never brought us complimentary noodles, i...",SERVICE,NaN,0,0,negative
3,1004293,1004293:3,The food was lousy - too sweet or too salty an...,FOOD,food,4,8,negative
4,1004293,1004293:3,The food was lousy - too sweet or too salty an...,FOOD,portions,52,60,negative


In [3]:
df_train.dropna(subset=['category', 'polarity','sentence'],inplace=True)

## Encode data

In [4]:
label_encoder = LabelEncoder()
train = Bunch()
test = Bunch()

train.sentence = df_train.sentence
test.sentence = df_test.sentence
train.cat = label_encoder.fit_transform(df_train.category.astype('category'))
test.cat = label_encoder.transform(df_test.category.astype('category'))
polarity2code = {'negative':-1, 'neutral':0, 'positive':1}
train.polarity = df_train.polarity.map(polarity2code)
test.polarity = df_test.polarity.map(polarity2code)

## Fit and Test 

In [5]:
tfidf_encoder = TfidfVectorizer(stop_words='english', strip_accents='unicode',min_df=2, max_df=0.98, max_features=1000)
svm_aspect = make_pipeline(tfidf_encoder,LinearSVC())
svm_aspect.fit(train.sentence,train.cat)

report = ClassificationReport()
test_scores_aspect = report.classifiction_report(svm_aspect,test.sentence,test.cat,threshold=0.2)
test_scores_aspect.round(decimals=3)

,accuracy,precision,recall,f1_score,auc,average_precision_score
0,0.698,0.661,0.561,0.607,0.079,0.645
1,0.698,0.571,0.316,0.407,0.185,0.467
2,0.698,0.776,0.806,0.790,0.111,0.861
3,0.698,0.375,0.231,0.286,0.117,0.291
4,0.698,0.603,0.643,0.622,0.149,0.642
5,0.698,0.673,0.690,0.682,0.089,0.758
micro,NaN,0.698,0.698,0.698,0.920,0.762


In [6]:
svm_polar = make_pipeline(tfidf_encoder,LinearSVC())
svm_polar.fit(train.sentence,train.polarity)

test_scores_polar = report.classifiction_report(svm_polar,test.sentence,test.polarity)
test_scores_polar.round(decimals=3)

,accuracy,precision,recall,f1_score,auc,average_precision_score
-1,0.749,0.510,0.598,0.551,0.181,0.557
0,0.749,1.000,0.114,0.204,0.190,0.351
1,0.749,0.839,0.845,0.842,0.157,0.930
micro,NaN,0.749,0.749,0.749,0.907,0.842


# Inspection

In [7]:
import eli5
from eli5.lime import TextExplainer

In [8]:
eli5.show_weights(svm_aspect, top=10, target_names=label_encoder.classes_)

In [9]:
eli5.show_weights(svm_polar, top=10, target_names=list(polarity2code.keys()))

In [10]:
i = 300
print('True Aspect:',label_encoder.inverse_transform(test.cat[i]))
print('True Polarity:',test.polarity[i])
print('Detected Polarity:',svm_polar.predict(test.sentence.iloc[[i]]))
print('Detected Aspect:')
eli5.show_prediction(svm_aspect.named_steps['linearsvc'],
                     test.sentence[i],
                     vec= svm_aspect.named_steps['tfidfvectorizer'],
                     top_targets=1, 
                     target_names=label_encoder.classes_ ,
                     top=4,
                     show_feature_values=True
                    )

True Aspect: SERVICE
True Polarity: 1
Detected Polarity: [1]
Detected Aspect:


/home/augustinus/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [20]:
print('Detected Polarity')
eli5.show_prediction(svm_polar.named_steps['linearsvc'],
                     test.sentence[355],
                     vec= svm_aspect.named_steps['tfidfvectorizer'],
                     top_targets=1, 
                     target_names=list(polarity2code.keys()) ,
                     top=10,
                     #show_feature_values=True,
                     force_weights=True,
                    )

Detected Polarity


# Apect detection as a multi-label classification problem 

In [14]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report,f1_score

## From a multi-class problem to a multi-label problem 

In multi-class classification formulation, we assume that each sentence belongs to exactly one aspect category. 

By reformulating it as multi-label classification problem, each sentence can belong to many aspect categories at the same time.

For example: the 3rd sentence in review 1004293 has two categories

In [15]:
df_train = pd.read_csv('./Restaurants/sub_task_1/ABSA16_Restaurants_Train_SB1_v2.csv')
df_test = pd.read_csv('./Restaurants/sub_task_1/EN_REST_SB1_TEST_gold.csv')
df_train['category'] = df_train['category'].str.split('#').str[0]
df_test['category'] = df_test['category'].str.split('#').str[0]

def collate_categories(df):
    grp = df.groupby('sid')
    categories = grp['category'].apply(lambda x: x.str.cat(sep=',')).str.split(',')
    categories_grouped =grp[['rid','sid','sentence']].head(1).join(categories,on='sid')
    return categories_grouped.reset_index(drop=True)
df_train = collate_categories(df_train)
df_test = collate_categories(df_test)

In [16]:
mlb = MultiLabelBinarizer(sparse_output=True)
train.sentence = df_train.sentence  
train.cat = mlb.fit_transform(df_train.category)
test.sentence = df_test.sentence  
test.cat = mlb.transform(df_test.category)

In [17]:
svm_multi_aspect = make_pipeline(tfidf_encoder,OneVsRestClassifier(LinearSVC(C=1.5),n_jobs=-1))
svm_multi_aspect.fit(train.sentence,train.cat)
y_pred = svm_multi_aspect.predict(test.sentence)

In [18]:
print(classification_report(test.cat,y_pred))
print('micro avg f1:',f1_score(test.cat,y_pred,average='micro'))

             precision    recall  f1-score   support

          0       0.88      0.65      0.75        57
          1       0.68      0.43      0.53        35
          2       0.84      0.80      0.82       257
          3       1.00      0.31      0.47        13
          4       0.68      0.59      0.63       193
          5       0.89      0.80      0.84       145

avg / total       0.81      0.70      0.75       700

micro avg f1: 0.7513389441469014


In [19]:
i = 355
print('True Aspect:',mlb.inverse_transform(test.cat[i]))
print('Detected Aspect:',mlb.inverse_transform(y_pred[i]))
eli5.show_prediction(svm_multi_aspect.named_steps['onevsrestclassifier'],
                     test.sentence[i],
                     vec= svm_multi_aspect.named_steps['tfidfvectorizer'],
                     top_targets=2, 
                     target_names=mlb.classes_ ,
                     top=10,
                     #show_feature_values=True,
                     force_weights=True,
                     #show=['WEIGHTS','targets','transition_features',
                     #                         'feature_importances','INFO','ALL']
                    )

True Aspect: [('FOOD', 'SERVICE')]
Detected Aspect: [('FOOD', 'SERVICE')]
